# TEM Image Processing Pipeline

In this notebook we implement an image processing pipeline to extract information from transmission electron microscopy images of barium titanate nanoparticles in a low density polyethylene _matrix_. Information gained from this process includes the:
- number of particles, $N_p$
- number of agglomerates, $N_a$
- size of particles (longest and shortest chord lengths)
- locations of particles (x,y)
- number of particles per agglomerate, $n$
- factor of agglomeration

where factor of agglomeration is given by:
$$F_a = \frac{N_a \cdot n}{N_p}$$

## 1. Import necessary libraries and setup

In [1]:
# for timing how long it takes to import libraries
import time
t_lib_init = time.perf_counter()

import cv2 as cv                   # OpenCV for image processing
import matplotlib.pyplot as plt    # Matplotlib for visualizing
import numpy as np                 # NumPy for quick maths
from collections import Counter    # dictionary quick maths
from tem_functions import *        # import necessary functions from separate python file

t_lib_final = time.perf_counter()

print("time to import libraries:", str(t_lib_final - t_lib_init), "seconds")

time to import libraries: 0.9006952080000001 seconds


Define constants

In [2]:
# In TES-II-36a.tif there are 113 pixels per 100 nm
# In Challenge_1.jpg there are 92 pixels per 200nm = 46 pixels per 100 nm

t_pipe_init = time.perf_counter()

# nm_per_pixel = 100 / 46
# nm_per_pixel = 100 / 95
nm_per_pixel = 100 / 113
expected_radius = 100
print(str(nm_per_pixel), "nm per pixel")

0.8849557522123894 nm per pixel
